In [28]:
import pandas as pd
import numpy as np

In [60]:
df = pd.read_excel(r"C:\Users\rampaln\OneDrive - NIWA\Research Projects\CAOA2101\week-it-snowed-everywhere\data\2. MasterWinterOutputTables_v2.xlsx",
 sheet_name="3-WinterOutput-60rs")
df2 = pd.read_excel(r"C:\Users\rampaln\OneDrive - NIWA\Research Projects\CAOA2101\week-it-snowed-everywhere\data\3. MasterAutumnOutputTables_v2.xlsx",
 sheet_name="12-AutumnOutput-60rs")

In [61]:
concat_data = pd.concat([df, df2],
                         ignore_index = True)

In [64]:
df2

,Subject ID,File name,Image name,Image segment,Month,Date,Attached thermometer (oF),Pass rate (%),No. Entries,Entry retire?,...,Force,Pass rate (%).6,No. Entries.6,Entry Retire?.4,Run,Pass rate (%).7,No. Entries.7,Entry Retire?.5,Segment retire?.2,Image retire?
0,41955610,A42581,A42581_1939_01_Mar_s1,1,3,1939-03-01,-999.0,1.0,43,1,...,5.0,1.0,32,1,-999.0,1.0,32,1,1,1
1,41955610,A42581,A42581_1939_01_Mar_s1,1,3,1939-03-02,-999.0,1.0,43,1,...,7.0,1.0,32,1,-999.0,1.0,32,1,1,1
2,41955610,A42581,A42581_1939_01_Mar_s1,1,3,1939-03-03,-999.0,1.0,43,1,...,5.0,1.0,32,1,-999.0,1.0,32,1,1,1
3,41955610,A42581,A42581_1939_01_Mar_s1,1,3,1939-03-04,-999.0,1.0,43,1,...,5.0,1.0,32,1,-999.0,1.0,32,1,1,1
4,41955610,A42581,A42581_1939_01_Mar_s1,1,3,1939-03-05,-999.0,1.0,43,1,...,4.0,1.0,32,1,-999.0,1.0,32,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5883,41954495,I68431,I68431_1939_05_May_s6,6,5,1939-05-27,-999.0,1.0,20,1,...,2.0,1.0,20,1,-999.0,1.0,20,1,1,1
5884,41954495,I68431,I68431_1939_05_May_s6,6,5,1939-05-28,-999.0,1.0,20,1,...,2.0,1.0,20,1,-999.0,1.0,20,1,1,1
5885,41954495,I68431,I68431_1939_05_May_s6,6,5,1939-05-29,-999.0,1.0,20,1,...,-999.0,1.0,20,1,-999.0,1.0,20,1,1,1
5886,41954495,I68431,I68431_1939_05_May_s6,6,5,1939-05-30,-999.0,1.0,20,1,...,4.0,1.0,20,1,-999.0,1.0,20,1,1,1


In [9]:
stations_list = pd.read_table(r"C:\Users\rampaln\OneDrive - NIWA\Python Tools\python_clidb\lib\CLIDB\database_metadata\station_list.txt")
stations_list

,Agent,Network,Start_Date,End_Date,Percent_Complete,Name,Lat(dec_deg),Long(dec_deg)
0,6202,L05300,8-Oct-85,5-Mar-20,50,Base Jubany,-62.233,-58.633
1,6207,L06600,26-Feb-99,5-Mar-20,40,Gen San Martin,-68.133,-67.133
2,5952,J56800,1-Jan-81,4-Mar-20,80,"Vanuatu,Aneityum",-20.233,169.767
3,5948,J55500,1-Jan-81,4-Mar-20,80,"Vanuatu,Lamap",-16.417,167.800
4,5946,J55100,1-Jan-81,4-Mar-20,80,"Vanuatu,Sola",-13.850,167.550
...,...,...,...,...,...,...,...,...
3672,4717,H31662,01-Aug-1888,28-Feb-1898,80,"Methven,Drayton",-43.600,171.617
3673,2622,D15133,01-Nov-1883,31-Oct-1888,80,Featherston,-41.117,175.333
3674,3389,E14276,01-Nov-1868,31-Oct-1869,100,"Wellington, Bowen St",-41.283,174.783
3675,3383,E14270,01-Feb-1862,31-Oct-1868,100,"Wellington, Knowles Observatory",-41.283,174.783


In [223]:
times = pd.date_range("1939-01-01", periods=500, freq ='1D')
dset_temp = pd.DataFrame(index = times)
dset_pressure = pd.DataFrame(index = times)
networks = concat_data['File name'].unique()


In [132]:
    agent = stations_list[stations_list.Network == site].Agent
    agent

1606    4532
Name: Agent, dtype: int64

In [224]:
for site in networks:
    subset = concat_data[concat_data['File name'] == site]
    subset.index = pd.to_datetime(subset['Date'])
    # get the coordsinates
    temp = subset['Attached thermometer (oF)']
    temp = temp[(temp> -999) & (temp <100)]
    pressure = subset['Barometer uncorrected']
    pressure = pressure[(pressure > -999) & (pressure < 1040)]

    pressure[(pressure>25.4)& (pressure< 31.0)] = pressure[(pressure>25.4)& (pressure< 31.0)] * 33.86
    pressure = pressure[pressure > 960]
    if (pressure.mean() < 998)|(len(pressure.dropna())< 30):
        pressure =[]
    agent = stations_list[stations_list.Network == site].Agent
    if len(agent) >0:
        if len(temp)>0:
            print(temp)
            dset_temp[site] = temp
        if len(pressure)>0:
            dset_pressure[site] = pressure


Date
1939-07-21    50.0
1939-07-22    53.0
1939-07-23    54.0
1939-07-24    52.5
1939-07-25    52.0
              ... 
1939-05-27    60.5
1939-05-28    59.0
1939-05-29    58.0
1939-05-30    59.0
1939-05-31    61.0
Name: Attached thermometer (oF), Length: 183, dtype: float64
Date
1939-08-01    47.4
1939-08-02    51.3
1939-08-03    52.4
1939-08-04    52.3
1939-08-05    52.2
1939-08-06    55.3
1939-08-07    53.4
1939-08-08    59.0
1939-08-09    58.0
1939-08-10    59.5
1939-08-11    59.0
1939-08-12    54.5
1939-08-13    53.0
1939-08-14    57.0
1939-08-15    59.0
1939-08-16    58.0
1939-08-17    61.0
1939-08-18    55.0
1939-08-19    54.0
1939-08-20    58.0
1939-08-21    57.0
1939-08-22    57.0
1939-08-23    61.0
1939-08-24    54.5
1939-08-25    57.5
1939-08-26    55.0
1939-08-27    56.5
1939-08-28    60.0
1939-08-29    61.0
1939-08-30    63.5
1939-08-31    59.0
Name: Attached thermometer (oF), dtype: float64
Date
1939-06-01    43.0
1939-06-02    41.0
1939-06-03    51.0
1939-06-04    45.0
19

In [230]:
    dset_pressure.isna().sum()

A64871    317
A42581    469
B75381    316
B75571    316
D05960    317
D87692    316
E14272    316
F20791    316
F38961    326
F47691    316
G13231    316
G22581    316
H31562    316
H31971    319
H32552    316
H32561    317
H32641    316
I50771    316
I59161    316
I68093    316
A53651    408
dtype: int64

In [237]:
sites = (dset_temp.isna().sum() < 318) & (dset_pressure.isna().sum() < 318)
temps = dset_temp[sites[sites == True].index]
pressure = dset_pressure[sites[sites == True].index]
pressure.dropna()

,A64871,B75381,B75571,F20791,F47691,G13231,G22581,H32561,H32641
1939-03-01,1021.52234,1010.85644,1020.8790,1027.38012,1025.41624,1025.34852,1025.9580,1031.64648,1030.22436
1939-03-02,1023.92640,1010.72100,1022.5720,1028.19276,1027.34626,1025.41624,1019.5246,1025.58554,1023.31692
1939-03-03,1024.50202,1014.37788,1023.2492,1028.59908,1027.65100,1025.68712,1021.2176,1026.60134,1025.34852
1939-03-04,1025.14536,1015.66456,1024.2650,1028.80224,1027.75258,1027.92188,1025.9580,1031.84964,1032.25596
1939-03-05,1025.34852,1015.73228,1023.9264,1029.81804,1028.22662,1027.17696,1022.9106,1029.20856,1027.65100
...,...,...,...,...,...,...,...,...,...
1939-08-27,998.26052,987.01900,998.1928,998.87000,999.27632,996.77068,995.1454,1002.59460,1001.84968
1939-08-28,996.83840,989.05060,998.8700,998.05736,999.68264,998.26052,994.4682,1002.39144,1000.56300
1939-08-29,1000.35984,989.79552,1000.9016,1000.42756,993.52012,998.46368,990.7436,997.54946,995.00996
1939-08-30,993.65556,983.29440,997.5156,984.24248,1015.12280,981.94000,975.5066,979.02804,978.48628


,A64871,B75381,B75571,F20791,F47691,G13231,G22581,H32561,H32641
A64871,1.000000,0.984209,0.964799,0.937712,0.871703,0.951298,0.895228,0.861663,0.849413
B75381,0.984209,1.000000,0.971924,0.931913,0.859275,0.954625,0.901177,0.864352,0.850444
B75571,0.964799,0.971924,1.000000,0.923594,0.860797,0.941069,0.889636,0.857816,0.846525
F20791,0.937712,0.931913,0.923594,1.000000,0.961126,0.990385,0.976304,0.963152,0.956180
F47691,0.871703,0.859275,0.860797,0.961126,1.000000,0.938980,0.949818,0.946612,0.945170
G13231,0.951298,0.954625,0.941069,0.990385,0.938980,1.000000,0.976203,0.960215,0.952340
G22581,0.895228,0.901177,0.889636,0.976304,0.949818,0.976203,1.000000,0.985939,0.982284
H32561,0.861663,0.864352,0.857816,0.963152,0.946612,0.960215,0.985939,1.000000,0.992351
H32641,0.849413,0.850444,0.846525,0.956180,0.945170,0.952340,0.982284,0.992351,1.000000


In [87]:
    subset = concat_data[concat_data['File name'] == site]
    subset.index = pd.to_datetime(subset['Date'])
    temp = subset['Attached thermometer (oF)']
    temp.min()

-999.0